In [95]:
import pandas as pd
import numpy as np
import random

In [57]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
combine = [train, test]

In [136]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,0,22,1,0,7.2500,1,1
1,2,1,1,1,38,1,0,71.2833,2,3
2,3,1,3,1,26,0,0,7.9250,1,2
3,4,1,1,1,35,1,0,53.1000,1,3
4,5,0,3,0,35,0,0,8.0500,1,1
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27,0,0,13.0000,1,5
887,888,1,1,1,19,0,0,30.0000,1,2
888,889,0,3,1,15,1,2,23.4500,1,2
889,890,1,1,0,26,0,0,30.0000,2,1


In [58]:
for data in combine:
    data['Sex'] = data['Sex'].apply(lambda x: 1 if x=='female' else 0)


In [109]:
train.info()
print('-'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    int32  
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Fare         891 non-null    float64
 9   Embarked     891 non-null    int64  
dtypes: float64(1), int32(1), int64(7), object(1)
memory usage: 66.3+ KB
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   

In [67]:
embark = {'S':1, 'C':2, 'Q':3}
def map_encode(val):
    return embark[val]

In [84]:
fill = train['Embarked'].dropna().mode()[0]

In [89]:
for data in combine:
    data['Embarked'] = data['Embarked'].fillna(fill)

In [86]:
for data in combine:
    data['Embarked'] = data['Embarked'].apply(map_encode)

In [90]:
for data in combine:
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())

In [103]:
for data in combine:
    avg_age = data['Age'].mean()
    std_age = data['Age'].std()
    count = data['Age'].isnull().sum()
    rand_list = np.random.randint(avg_age-std_age, avg_age+std_age, size=count)
    data['Age'][np.isnan(data['Age'])] = rand_list
    data['Age'] = data['Age'].astype('int')

<ipython-input-103-262a80fc5919>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age'][np.isnan(data['Age'])] = rand_list


In [101]:
print(np.random.randint(avg_age-std_age, avg_age+std_age, size=5))

[22 29 28 19 20]


In [108]:
for data in combine:
    data.drop(['Ticket','Cabin'], axis=1, inplace=True)


In [36]:
df["SibSp"].unique()

array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

In [38]:
df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6], dtype=int64)

In [113]:
for data in combine:
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [124]:
test['Title'].value_counts()

Mr        240
Miss       79
Mrs        72
Master     21
Rare        6
Name: Title, dtype: int64

In [126]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for data in combine:
    data['Title'] = data['Title'].map(title_mapping)
    data['Title'] = data['Title'].replace(['Mlle','Ms'], 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [129]:
for data in combine:
    data.drop(['Name'], inplace=True, axis=1)

In [137]:
x_train = train.drop(['Survived'], axis=1)
y_train = train['Survived']

In [138]:
x_test = test

In [133]:
from sklearn.linear_model import LogisticRegression

In [161]:
clf = LogisticRegression(solver = 'liblinear', penalty='l1', max_iter=1000)

In [162]:
clf.fit(x_train, y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [163]:
clf.score(x_train, y_train)

0.8125701459034792

In [164]:
pred = clf.predict(x_test)

In [168]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [169]:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [170]:
cross_val_score(tree, x_train, y_train)

array([0.53631285, 0.80337079, 0.76966292, 0.76966292, 0.7247191 ])